In [1]:
# import modules
import shutil
import os
from datetime import date

In [2]:
'''Copying and renaming files'''

def bidsify(origpath, destpath, n_sessions=2, scan_types=None, scan_names=None, log_changes=True, verbose=True):
    
    # instantiate list of unsuccessfully handled file paths
    problem_files = []
    
    if scan_types is None:
        scan_types = ['anat','func']
        
    if scan_names is None:
        # use default dictionary of scan types
        scan_names = {
            'mprage' : 'T1w',
            'bold1' : 'task-rest_run-01_bold',
            'bold2' : 'task-mcr_run-02_bold',
            'bold3' : 'task-swm_run-03_bold',
            'bold4' : 'task-dd_run-04_bold',
            'bold5' : 'task-rest_run-05_bold'
        }
        
    if log_changes:
        scan_types.append('log')
        
    
    for i, (root, dirs, files) in enumerate(os.walk(origpath)):
        # create new directory structure
        if i == 0:
            [os.makedirs(destpath+'/'+direc.split('_')[0]+'/ses-'+str(ses+1)+'/'+scantype, exist_ok=True) for direc in dirs for scantype in scan_types for ses in range(n_sessions)]
        
        # move and rename files
        else:
            file_list = [f for f in files if not f.startswith('.')]
            if file_list:
                for file in file_list:
                    old_filepath = os.path.join(root, file)
                    new_filepath, problem_file = _rename(file, root, n_sessions, scan_names, destpath)
                    
                    if new_filepath is not None:
                        # move and rename
                        shutil.copy(old_filepath, new_filepath)
                        
                    else:
                        problem_files.append(problem_file)
                        
                        
                    if problem_files:
                        print('The following files were not successfully converted: ' + problem_files)

In [9]:
'''Defines naming scheme for moved files'''

def _rename(file, root, n_sessions, scan_names, destpath):

    # viarable to track unsuccessfully renamed files
    problem_file = None

    old_path = os.path.join(root, file)
    base, ext = os.path.splitext(file)
    splitpath = root.split('/')

    # get subject ID
    sub = splitpath[-2].split('_')[0]

    # get session number
    ses_number = splitpath[-2].split('_')[1]
    if int(ses_number) <= n_sessions:
        session = 'ses-'+ses_number

    else:
        print('unrecognized session number \'' + ses_number + '\' for subID ' + sub)
        problem_file = file

    # get scan type (or log)
    if splitpath[-1] == 'ANATOMY':
        runtype = 'anat'

    elif splitpath[-1] == 'FUNCTIONAL':
        runtype = 'func'

    elif splitpath[-1] == 'LOG':
        runtype = 'LOG'

    else:
        print('unrecognized scan or log folder ' + splitpath[-1] + ' for subID ' + sub)
        problem_file = file

    # format scan name (or preserve name of log file)
    if base in scan_names:
        new_name = scan_names[base]

    elif ext =='.log':
        new_name = base

    else:
        print('unrecognized scan name ' + base + ' for file ' + file)
        problem_file = file

    try:
        if runtype != 'LOG':
            new_path = os.path.join(destpath, sub, session, runtype, sub+'_'+new_name+ext)
        else:
            new_path = os.path.join(destpath, sub, session, runtype, new_name+ext)

    except NameError:
        new_path = None


    if new_path is not None:
        _writelog(old_path, destpath, sub, session, new_name, ext)
    
    return new_path, problem_file

In [10]:
'''Writes/updates log of moving file'''

def _writelog(old_path, destpath, sub, session, new_name, ext):
    
    filename = os.path.join(destpath, sub, session, 'LOG/CHANGES.log')
    
    if os.path.exists(filename):
        mode = 'a'
    
    else:
        mode = 'w'
        
    with open(filename, mode) as f:
        f.write(
            date.today().strftime('%Y-%m-%d') + '\n' +
            ' - ' + new_name+ext + ' moved from ' + old_path + '\n'
        )

In [11]:
'''TESTING'''


bidsify('/Users/paxtonfitzpatrick/Desktop/testorig/', '/Users/paxtonfitzpatrick/Desktop/testdest/', log_changes=True)